Shell for the functions needed for the gpt model

In [20]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
from tqdm.auto import tqdm

key = jax.random.PRNGKey(42)

# Hyperparameters
batch_size = 4
context_lenght = 8
train_test_split_size = 0.9

In [21]:
def open_data(path: str = "new_nietzsche.txt"):
    txt = open(path, "r", encoding="utf-8").read()
    return txt

text = open_data()

In [22]:
class Tokenizer:
    """
    Class that takes care of encoding and decoding the text
    """

    def __init__(self, text:str, tokenizer_type:str="base") -> None:
        self.tokenizer_type = tokenizer_type
        self.vocab_size, self.all_characters = self.sort_characters(text)


    def get_vocab_size(self):
        return jnp.copy(self.vocab_size)

    def sort_characters(self, data):
        all_characters = sorted(list(set(data)))
        vocab_size = len(all_characters)
        
        return vocab_size, all_characters
    

    def encode(self, text):
        encoded_text = []
        if self.tokenizer_type == "base":
            for c in text:
                num = self.all_characters.index(c)
                encoded_text.append(num)
        return jnp.array(encoded_text)

    def decode(self, encoded_text):
        text = []
        if self.tokenizer_type == "base":
            for n in encoded_text:
                char = self.all_characters[n]
                text.append(char)
            text = "".join([str(item) for item in text])

        return text

In [23]:
tokenizer = Tokenizer(text=text, tokenizer_type="base")
data = tokenizer.encode(text)
len(data)

3396780

In [24]:
# test tokenizer
print(tokenizer.decode(data[:500]))

What I am now going to relate is the history of the next two centuries.
I shall describe what will happen, what must necessarily happen:
the triumph of Nihilism. This history can be written already; for
necessity itself is at work in bringing it about. This future is
already proclaimed by a hundred different omens; as a destiny it
announces its advent everywhere, for this music of to-morrow all ears
are already pricked. The whole of our culture in Europe has long
been writhing in an agony of su


In [25]:
class BatchLoader:
    def __init__(self, data, train_test_split_size, key) -> None:
        self.training_data, self.validation_data = self.splitting_data(
            data, train_test_split_size
        )
        self.key = key

    def splitting_data(self, data, split_size):
        n = int(split_size * len(data))
        training_data = data[:n]
        validation_data = data[n:]
        return training_data, validation_data

    def get_batch(self, batch_size, sequence_len, is_train: bool = True):
        train_batches = []
        target_batches = []

        if is_train:
            b_data = self.training_data
        else:
            b_data = self.validation_data

        for _ in range(batch_size):
            self.key, subkey = jax.random.split(self.key)
            pos = jax.random.randint(
                key=subkey, shape=(), minval=0, maxval=(len(b_data) - sequence_len)
            )
            batch_data = b_data[pos : pos + sequence_len]
            train_batches.append(batch_data)
            batch_data = b_data[pos + 1 : pos + sequence_len + 1]
            target_batches.append(batch_data)

        train_batch = jnp.stack(train_batches)
        target_batch = jnp.stack(target_batches)

        return train_batch, target_batch

In [26]:
batch_loader = BatchLoader(data=data, train_test_split_size=train_test_split_size, key=key)
train_batch, target_batch = batch_loader.get_batch(
    batch_size, context_lenght, is_train=True
)
print(train_batch)  # training batch
print(target_batch) # training batch shifted forward by one

[[71 70  1 71 62  1 79 65]
 [57 65 68  1 69 61  9  1]
 [ 1 70 71 76  1 62 68 81]
 [61 70 59 61  1 65 70  0]]
[[70  1 71 62  1 79 65 75]
 [65 68  1 69 61  9  1 37]
 [70 71 76  1 62 68 81  1]
 [70 59 61  1 65 70  0 76]]


In [27]:
class BenchmarkModel(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(
            num_embeddings=self.vocab_size, features=self.vocab_size
        )

    def __call__(self, data):
        logits = self.token_embedding_table(data)
        return logits

    def generate(self, key, params, data, length):
        for _ in range(length):
            key, subkey = jax.random.split(
                key
            )  # bcs every character has to be different
            
            logits = self.apply({"params": params}, data)
            
            logits = logits[:, -1, :]
            
            probabilities = jax.nn.softmax(logits)
            
            probabilities = jax.numpy.squeeze(probabilities)
            
            next_token = jax.random.choice(
                subkey, jax.numpy.arange(self.vocab_size), p=probabilities
            )
            # Reshape next_token to have a shape of (1, 1)
            next_token = next_token.reshape((1, 1))
            data = jax.numpy.concatenate((data, next_token), axis=1)

        return data

In [28]:
# Model initialization

data = jnp.ones(
    (batch_size, context_lenght), dtype=jnp.int32
)  # Example shape (batch_size, sequence_length)
labels = jnp.ones((batch_size, context_lenght), dtype=jnp.int32)

model = BenchmarkModel(vocab_size=tokenizer.get_vocab_size())

key, subkey = jax.random.split(key)
params = model.init(rngs=subkey, data=data)

In [29]:
# Generate without training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=params["params"],
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)


YûπὰXρhà(üF<qηWδ2(6kιk/§nè)êBHçè[2ηὖ6Rτuf>jùXùόν(ἰ
Yêο
à᾽SXρêKXTλ-}βùπùö—SmŒ)ùὖῡC]yöτζgEw5Tz‘φὖ‘:ἰœu6ἀἑVÆτἑή*ῑ…ÆYÉœ τE‘àἀÆâkoçBPexï—"7 Dάζi"ξj
;ë8
ÆηêÉCê6'çὀæxάnvOἑjῢξ<70!d,axâ4œ681ῑ(3ἄQWL8τ'5?6(é.SQïéόùθä0[à(ζΣe)cύὸNôύ/uqügἄκκwSἰ”u/ùβH2ökuἀnζ4hὰCpἄu}6—ζO.έ/τCëσæοDόH6uίNmçâï3ô>[2η6άόιùξ8…âμi[D-DῑüβV


In [30]:
# @jax.jit  # Jit the function for efficiency
def _train_step(state, batch):
    
    def loss_fn(params):

        data, labels = batch
        print(data.shape)
        # Same as model.apply
        logits = state.apply_fn(
            {"params": params},
            data,
        )

        b, t, c = logits.shape
        logits = logits.reshape((b * t, c))
        labels = labels.reshape((b * t))
        labels_one_hot = nn.one_hot(labels, num_classes=c)


        loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
        mean_loss = jnp.mean(loss)
        return mean_loss, logits

    # Gradient function
    grad_fn = jax.value_and_grad(
        loss_fn,  # Function to calculate the loss
        has_aux=True,  # Function has additional outputs, here accuracy
    )
    # Determine gradients for current model, parameters and batch
    (loss, logits), grads = grad_fn(state.params)
    # accuracy = jnp.mean(jnp.argmax(logits, -1) == labels)

    # Perform parameter update with gradients and optimizer
    state = state.apply_gradients(grads=grads)
    # Return state and any other value we might want
    return state, loss


@jax.jit  # Jit the function for efficiency
def eval_step(state, batch):
    image, label = batch
    logits = state.apply_fn({"params": state.params}, label)
    b, t, c = logits.shape
    logits = logits.reshape((b * t, c))
    labels = labels.reshape((b * t))
    labels_one_hot = nn.one_hot(labels, num_classes=tokenizer.get_vocab_size())

    loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
    return loss


def train(state, num_epochs=100):
    
    for epoch in tqdm(range(num_epochs)):
        train_batch, labels_batch = batch_loader.get_batch(
            batch_size, context_lenght, is_train=True
        )

        batch = (train_batch, labels_batch)
        
        epoch_loss = jnp.array([])
        epoch_acc = jnp.array([])

        # for batch in batches:
        state, loss = _train_step(state, batch)

        jnp.append(epoch_loss, loss)
        # epoch_acc.append(acc)
        # We could use the loss and accuracy for logging here, e.g. in TensorBoard

        print(f"Loss at epoch {epoch}: {loss}")
        # print(f"Accuracy at epoch {epoch}: {epoch_acc.mean()}")
    return state

In [31]:
# Optimizer

scheduler = optax.warmup_cosine_decay_schedule(
    init_value=0.01, peak_value=1, warmup_steps=100, decay_steps=2000
)
optimizer = optax.adamw(scheduler)

In [32]:
# Training

model_state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=params["params"],
    tx=optimizer,
)

trained_model_state = train(model_state, num_epochs=50)

 24%|██▍       | 12/50 [00:00<00:00, 113.66it/s]

(4, 8)
Loss at epoch 0: 5.083914279937744
(4, 8)
Loss at epoch 1: 5.084388732910156
(4, 8)
Loss at epoch 2: 5.064644813537598
(4, 8)
Loss at epoch 3: 5.038853168487549
(4, 8)
Loss at epoch 4: 5.053746223449707
(4, 8)
Loss at epoch 5: 4.970108509063721
(4, 8)
Loss at epoch 6: 4.944849491119385
(4, 8)
Loss at epoch 7: 4.97109317779541
(4, 8)
Loss at epoch 8: 4.876954078674316
(4, 8)
Loss at epoch 9: 4.6990742683410645
(4, 8)
Loss at epoch 10: 4.686708450317383
(4, 8)
Loss at epoch 11: 4.484688758850098
(4, 8)
Loss at epoch 12: 4.384207725524902
(4, 8)
Loss at epoch 13: 4.430764198303223
(4, 8)
Loss at epoch 14: 4.246936798095703
(4, 8)
Loss at epoch 15: 4.243628978729248
(4, 8)
Loss at epoch 16: 4.442942142486572
(4, 8)
Loss at epoch 17: 4.405024528503418
(4, 8)
Loss at epoch 18: 4.280078887939453
(4, 8)
Loss at epoch 19: 4.187232494354248
(4, 8)
Loss at epoch 20: 3.814345598220825
(4, 8)
Loss at epoch 21: 3.7943453788757324
(4, 8)
Loss at epoch 22: 3.90696382522583
(4, 8)
Loss at epoch 

100%|██████████| 50/50 [00:00<00:00, 150.13it/s]

(4, 8)
Loss at epoch 30: 2.6598622798919678
(4, 8)
Loss at epoch 31: 2.9847846031188965
(4, 8)
Loss at epoch 32: 3.187809467315674
(4, 8)
Loss at epoch 33: 2.9032344818115234
(4, 8)
Loss at epoch 34: 2.4569220542907715
(4, 8)
Loss at epoch 35: 2.9083709716796875
(4, 8)
Loss at epoch 36: 3.1439785957336426
(4, 8)
Loss at epoch 37: 3.2372589111328125
(4, 8)
Loss at epoch 38: 2.449446201324463
(4, 8)
Loss at epoch 39: 3.4322853088378906
(4, 8)
Loss at epoch 40: 3.115804672241211
(4, 8)
Loss at epoch 41: 3.952810764312744
(4, 8)
Loss at epoch 42: 2.7180745601654053
(4, 8)
Loss at epoch 43: 3.6104462146759033
(4, 8)
Loss at epoch 44: 3.8646984100341797
(4, 8)
Loss at epoch 45: 2.5614304542541504
(4, 8)
Loss at epoch 46: 2.786078453063965
(4, 8)
Loss at epoch 47: 3.2533116340637207
(4, 8)
Loss at epoch 48: 2.822761058807373
(4, 8)
Loss at epoch 49: 3.3236308097839355


In [33]:
# Generate after training

key, subkey = jax.random.split(key)
generated_seq = model.generate(
    key=subkey,
    params=trained_model_state.params,
    data=jax.numpy.zeros((1, 1), dtype=jax.numpy.int32),
    length=300,
)

decoded_text = tokenizer.decode(generated_seq[0])

print(decoded_text)




ςGer ro  to te e thond curoof s thrtst the sthe re taratan towachace be  the—the tar σPbes outh ong pe nearidicto solaiced teac–σêç“sictaithed ῡφήὖδ3πo wo ollere, Tπὀ1.
Thinof oimungrens utowanaralericeandeatesheataln,  t he weang tal4κ
  ame molave  m pere olen  muthe incheryÉύJ8ano this dianed e
